## 本程序移除不满足要求的位点信息，服务对象是已经挑选出cluster、expWT等文件到"_treat"文件夹的数据。本程序不统计label值。

### 假设需要统计的文件的目录结构为“AuPd-4000_au150-0_WT_untar_treat”（一级目录）——“500”（二级目录）——“cluster/coor/expWT”（三级目录），将本程序安放在与一级目录平行的另一个一级目录（或一级目录本身）的任意一个二级文件目录内，即可运行。
### 它会将不符合的数据，提取到与“cluster、coor、expWT”等平行目录的“check”文件夹中

last update: 2021.9.14

contacts: zhaohf@ihep.ac.cn

# 配置环境

## 导入各种模块

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import shutil

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
print('numpy version:',np.__version__)
print('panda version:',pd.__version__)
#print('shutil version:',shutil.__version__)

python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.19.5
panda version: 1.2.4


# 设置输入文件夹和输出文件夹

## 设置源文件夹，即要处理的文件夹

In [3]:
# 设置源文件夹，即经过计算集群计算的文件夹
dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_treat'

## 设置移除的文件夹

In [4]:
dir_check = 'check'

## 设置认可的配位数范围

In [5]:
cn_max = 14
cn_min = 3

# 一些函数、类

## 读取coor.dat文件中的数组

In [6]:
def read_coor(path_coor, shell=1):
    """
    读取coor.dat中前shell个壳层的数据
    """
    exist = os.path.exists(path_coor)
    if exist:
        coor = pd.read_csv(path_coor,delim_whitespace=True,header=None, skiprows=[0,1])
        coor_data = coor.values
        cn = coor_data[0:shell,1]
        r = coor_data[0:shell,2]
        r_d = coor_data[0:shell,3]
#
        return cn,r,r_d
#
    else:
        return None  

# 程序正文

## 设置工作文件夹

In [7]:
# 当前目录
path = os.path.abspath('./')
path

'D:\\haifeng\\work_20210709\\ML+XAS\\code\\datasets setup\\jupyterbook'

In [8]:
# 上一级目录
path_par = os.path.abspath(os.path.pardir)
path_par

'D:\\haifeng\\work_20210709\\ML+XAS\\code\\datasets setup'

In [9]:
# 设置源文件夹路径
path_src = os.path.join(path_par, dir_src)

In [10]:
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT untared directory', dir_src, 'is not found!' )
    print()
#    sys.exit()
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900']

In [12]:
#  N_bad 表示不好的位点数量（也就是配位数超过m的位点数）
N_bad = 0
#
# 进入各个cluster，即100、200等
for dir_seed in list_seed:
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 先检查各seed下有没有coor文件夹，如果没有，则报错，因为这样无法进行判断
    dir_coor = 'coor'
    path_coor_src = os.path.join(path_seed_src, dir_coor)
    exist = os.path.exists(path_coor_src)
    if exist:
        pass
    else:
        print()
        print('** Error!! Directory', dir_coor, 'is not found!' )
#        sys.exit()
#
#  在源文件夹内，查一下有哪些文件夹
    list_choose = os.listdir(path_seed_src)
# 创建“check”文件夹目录
#    dir_check = 'check'
    path_check_work = os.path.join(path_seed_src, dir_check)
    os.makedirs(path_check_work, exist_ok=True)
#
# 源seed/coor中各个coor文件  
    list_coor = os.listdir(path_coor_src)
    for file_coor in list_coor:
#
# 各个coor_i.dat文件的路径
        path_coorN_src = os.path.join(path_coor_src, file_coor)
#
#  读取coor_i.dat中的信息
        cn,r,r_d = read_coor(path_coorN_src,1)
#
        if cn[0] < cn_min or cn[0] > cn_max :
            N_bad = N_bad + 1
            print()
            print('** file', dir_seed, '/', file_coor, ' is moved to ''check'' directory. **')
            print('** Its CN is', cn[0])
            print('** Its CR is', r[0])
            print('** Its CR delta is', r_d[0])
#
#  将配位数大于m的文件——所有seed文件夹下各文件夹中的文件，如coor、cluster、expWT等——移动到check文件夹下
            try:
                shutil.move(path_coorN_src,path_check_work)
            except shutil.Error:
                path_coorN_work = os.path.join(path_check_work, file_coor)
                os.remove(path_coorN_work)
                shutil.move(path_coorN_src,path_check_work)
#
#  对coorN.dat文件进行断句，找到是第几个位点，便于去cluster、expWT文件夹中进行寻找。
            result = os.path.splitext(file_coor)
            number = result[0][5:]
            key = '_' + number + '.'
            for dest in list_choose:
                if dest != 'coor' and dest != 'check':
                    path_dest_src = os.path.join(path_seed_src, dest)
                    list_dest = os.listdir(path_dest_src)
                    found = False
                    for file in list_dest:
                        if key in file:
                            found = True
                            break
                    if found:
                        path_file_src = os.path.join(path_dest_src, file)
                        try:
                            shutil.move(path_file_src,path_check_work)
                        except shutil.Error:
                            path_file_work = os.path.join(path_check_work, file)
                            os.remove(path_file_work)
                            shutil.move(path_file_src,path_check_work)
#
# 删除 空文件夹
#
    try:
        os.removedirs(path_check_work)
    except:
        pass
#
print()
print(f"** {N_bad} sites'cn < {cn_min} or > {cn_max} should be checked. **")
#


** 0 sites'cn < 3 or > 14 should be checked. **
